In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
from joblib import dump, load

In [2]:
data = pd.read_csv(os.path.join('Added_good_or_bad_8_column.csv'))
data.head()

,imdb_title_id,original_title,year,genre,duration,language,production_company,avg_vote,votes,budget,Good_or_Bad_Movie,number_of_genres,number_of_languages,first_genre,Good_or_Bad_Movie_6_or_higher,Good_or_Bad_Movie_8_or_higher
0,tt7939428,10 Things We Should Do Before We Break Up,2020,Romance,74,English,NaN,4.7,486,25000000,0,1,1,Romance,0,0
1,tt10740584,Abigail Haunting,2020,Horror,85,English,Indie Film Factory,3.8,398,60000,0,1,1,Horror,0,0
2,tt11032990,Malibu Rescue: The Next Wave,2020,"Action, Adventure, Comedy",68,English,Pacific Bay Entertainment,4.5,386,5000000,0,3,1,Action,0,0
3,tt8461224,The Tax Collector,2020,"Action, Crime, Drama",95,English,Cedar Park Entertainment,4.7,4862,30000000,0,3,1,Action,0,0
4,tt11547496,Camp Blood 8: Revelations,2020,Horror,82,English,Sterling Entertainment,2.2,114,10000,0,1,1,Horror,0,0


In [3]:
# Create a new dataframe with the columns needed for ML
data2 = data.drop(columns=['imdb_title_id','original_title','genre','language','production_company','first_genre','votes'])
data2.head()

,year,duration,avg_vote,budget,Good_or_Bad_Movie,number_of_genres,number_of_languages,Good_or_Bad_Movie_6_or_higher,Good_or_Bad_Movie_8_or_higher
0,2020,74,4.7,25000000,0,1,1,0,0
1,2020,85,3.8,60000,0,1,1,0,0
2,2020,68,4.5,5000000,0,3,1,0,0
3,2020,95,4.7,30000000,0,3,1,0,0
4,2020,82,2.2,10000,0,1,1,0,0


In [4]:
#Try to bin the budget, add a new column to the dataframe and run the model
cut_bins = [100000,1000000,10000000,100000000,1000000000,10000000000]
cut_labels = ["<100K","100K-1M","1M-10M","10M-100M","100M-1B"]
data2['budget_bin'] = pd.cut(data2['budget'], bins=cut_bins, labels=cut_labels)

In [5]:
# Budget_bin is a categorical variable, so split up the categorical variables determine if that leads to a better model
data3 = pd.get_dummies(data2)
data3.head()

,year,duration,avg_vote,budget,Good_or_Bad_Movie,number_of_genres,number_of_languages,Good_or_Bad_Movie_6_or_higher,Good_or_Bad_Movie_8_or_higher,budget_bin_<100K,budget_bin_100K-1M,budget_bin_1M-10M,budget_bin_10M-100M,budget_bin_100M-1B
0,2020,74,4.7,25000000,0,1,1,0,0,0,0,1,0,0
1,2020,85,3.8,60000,0,1,1,0,0,0,0,0,0,0
2,2020,68,4.5,5000000,0,3,1,0,0,0,1,0,0,0
3,2020,95,4.7,30000000,0,3,1,0,0,0,0,1,0,0
4,2020,82,2.2,10000,0,1,1,0,0,0,0,0,0,0


In [6]:
# Assign X (data) and y (target)
X = data3[["duration","budget_bin_<100K","budget_bin_100K-1M","budget_bin_1M-10M","budget_bin_10M-100M","budget_bin_100M-1B"]]
y = data3["Good_or_Bad_Movie_8_or_higher"]
print(X.shape, y.shape)

(10345, 6) (10345,)


In [191]:
#Split our data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=1)

## Try Logistic Model First

In [193]:
#Create a Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(class_weight="balanced")
#classifier = LogisticRegression()
classifier


LogisticRegression(class_weight='balanced')

In [194]:
#Fit (train) or model using the training data
classifier.fit(X_train, y_train)

C:\Users\ramya\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(class_weight='balanced')

In [195]:
#Validate the model using the test data
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.7954702389172766
Testing Data Score: 0.7957474226804123


In [196]:
# Need the duration and budget inputs as a list, then put into another list and call predict on it to get the predicted good or bad movie
#predictions = classifier.predict([[100,1000000,1,1]])
#predictions

In [197]:
#Make predictions
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [0 0 1 1 0 0 0 0 1 0]
First 10 Actual labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [198]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,0,0
1,0,0
2,1,0
3,1,0
4,0,0
...,...,...
3099,0,0
3100,0,0
3101,1,0
3102,0,0


## Try a Random Forest Model

In [202]:
# Instantiate model 2 (e.g., random forest model)
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200)
model.fit(X_train, y_train)
model_training_score = round(model.score(X_train, y_train)*100,3)
base_accuracy = round(model.score(X_test, y_test)*100,3)
print(f"Training Data Score: {model_training_score} %")
print(f"Testing Data Score: {base_accuracy} %")

Training Data Score: 98.315 %
Testing Data Score: 97.423 %


In [200]:
#Make predictions
predictions = model.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [0 0 0 0 0 0 0 0 0 0]
First 10 Actual labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [203]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
3099,0,0
3100,0,0
3101,0,0
3102,0,0


#### In an attempt to tune hyperparameters I have linked the resource I have used
https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

In [17]:

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# define models and parameters
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

C:\Users\ramya\anaconda3\envs\PythonAdv\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\ramya\anaconda3\envs\PythonAdv\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\ramya\anaconda3\envs\PythonAdv\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\ramya\anaconda3\envs\PythonAdv\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\ramya\anaconda3\envs\PythonAdv\lib\site-packages\scipy\optimize\linesearch.py:477: Line

Best: 0.828645 using {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.828645 (0.001545) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.828420 (0.000400) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.828420 (0.000400) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.828613 (0.001378) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.828420 (0.000400) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.828420 (0.000400) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.828452 (0.001209) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.828420 (0.000400) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.828420 (0.000400) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.828388 (0.001423) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.828420 (0.000400) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.828420 (0.000400) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.828452 (0.001227) wit

C:\Users\ramya\anaconda3\envs\PythonAdv\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\ramya\anaconda3\envs\PythonAdv\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\ramya\anaconda3\envs\PythonAdv\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\ramya\anaconda3\envs\PythonAdv\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\ramya\anaconda3\envs\PythonAdv\lib\site-packages\scipy\optimize\linesearch.py:477: Line